# CatBoost - 전체 데이터셋

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lg-aimers-ensemble/lgbmmmm.csv
/kaggle/input/lg-aimers-ensemble/xgb_submission.csv
/kaggle/input/lg-aimers/sample_submission.csv
/kaggle/input/lg-aimers/train.csv
/kaggle/input/lg-aimers/test.csv


In [5]:
data_path = '/kaggle/input/lg-aimers/'

train = pd.read_csv(data_path + 'train.csv', index_col= 'ID')
test = pd.read_csv(data_path + 'test.csv', index_col= 'ID')
submission = pd.read_csv(data_path + 'sample_submission.csv', index_col= 'ID')

## 1. 데이터 통합

In [6]:
all_data = pd.concat([train, test], ignore_index= True)
all_data = all_data.drop('임신 성공 여부', axis= 1)

In [7]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [8]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    all_data[col] = all_data[col].astype(str)

## 2. 피처 엔지니어링: 원 핫 인코딩

In [9]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[categorical_columns])

encoded_cat_matrix

<346418x205 sparse matrix of type '<class 'numpy.float64'>'
	with 16281646 stored elements in Compressed Sparse Row format>

In [10]:
all_columns = all_data.columns
all_columns

Index(['시술 시기 코드', '시술 당시 나이', '임신 시도 또는 마지막 임신 경과 연수', '시술 유형', '특정 시술 유형',
       '배란 자극 여부', '배란 유도 유형', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부',
       '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인',
       '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 난관 질환',
       '불임 원인 - 남성 요인', '불임 원인 - 배란 장애', '불임 원인 - 여성 요인', '불임 원인 - 자궁경부 문제',
       '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인',
       '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아 생성 주요 이유', '총 시술 횟수',
       '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수',
       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '총 생성 배아 수',
       '미세주입된 난자 수', '미세주입에서 생성된 배아 수', '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수',
       '미세주입 후 저장된 배아 수', '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
       '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수', '난자 출처', '정자 출처',
       '난자 기증자 나이', '정자 기증자 나이', '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부',
       '대리모 여부', 'PGD 시술 여부', 'PGS 시술 여부', '난자

In [11]:
remaining_columns = list(set(all_columns) - set(categorical_columns))

In [12]:
from scipy import sparse

extracted_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_columns]),
                               encoded_cat_matrix],
                              format='csr')

## 3. 데이터 나누기

In [13]:
num_train = len(train) 

X = extracted_data_sprs[:num_train]
X_test = extracted_data_sprs[num_train:]

y = train['임신 성공 여부'].values

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)

In [15]:
from catboost import CatBoostClassifier, Pool

# CatBoost 전용 데이터셋
bayes_dtrain = Pool(X_train, y_train, cat_features=None)
bayes_dvalid = Pool(X_valid, y_valid, cat_features=None)

## 4. 층화 K 폴드

In [16]:
from sklearn.model_selection import StratifiedKFold

# 층화 K 폴드 교차 검증기
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1991)

## 5. 베이지안 최적화

In [15]:
# 베이지안 최적화를 위한 하이퍼파라미터 범위
param_bounds = {
    'depth': (4, 10),
    'learning_rate': (0.01, 0.1),
    'l2_leaf_reg': (1, 10),
    'bagging_temperature': (0.1, 1.0),
    'border_count': (32, 255)
}

# 고정된 하이퍼파라미터
fixed_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'iterations': 2500,
    'random_seed': 1991,
    'verbose': 200
}

In [19]:
from sklearn.metrics import roc_auc_score

# 평가 함수 정의
def eval_function(depth, learning_rate, l2_leaf_reg, bagging_temperature, border_count):
    params = {
        'depth': int(round(depth)),
        'learning_rate': learning_rate,
        'l2_leaf_reg': l2_leaf_reg,
        'bagging_temperature': bagging_temperature,
        'border_count': int(round(border_count))
    }
    
    params.update(fixed_params)
    print('하이퍼파라미터:', params)
    
    # CatBoost 모델 훈련
    cat_model = CatBoostClassifier(**params)
    cat_model.fit(bayes_dtrain, 
                  eval_set=bayes_dvalid, 
                  early_stopping_rounds=100, 
                  verbose=0)
    
    # 검증 데이터 예측
    preds = cat_model.predict_proba(X_valid)[:, 1]
    
    # ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, preds)
    print(f'ROC-AUC : {roc_auc}\n')
    
    return roc_auc

In [20]:
from bayes_opt import BayesianOptimization

# 베이지안 최적화 객체 생성
optimizer = BayesianOptimization(f=eval_function, pbounds=param_bounds, random_state=0)

In [21]:
# 베이지안 최적화 수행
# TODO: init_points 10~15, n_iter 30~70
optimizer.maximize(init_points=10, n_iter=50)

|   iter    |  target   | baggin... | border... |   depth   | l2_lea... | learni... |
-------------------------------------------------------------------------------------
하이퍼파라미터: {'depth': 8, 'learning_rate': 0.048128931940501433, 'l2_leaf_reg': 5.903948646972072, 'bagging_temperature': 0.5939321535345923, 'border_count': 191, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'iterations': 2500, 'random_seed': 1991, 'verbose': 200}
ROC-AUC : 0.7414941350956854

| 1         | 0.7415    | 0.5939    | 191.5     | 7.617     | 5.904     | 0.04813   |
하이퍼파라미터: {'depth': 9, 'learning_rate': 0.04450973669432, 'l2_leaf_reg': 9.672964844509263, 'bagging_temperature': 0.6813047017599905, 'border_count': 130, 'loss_function': 'Logloss', 'eval_metric': 'AUC', 'iterations': 2500, 'random_seed': 1991, 'verbose': 200}
ROC-AUC : 0.7419289371810154

| 2         | 0.7419    | 0.6813    | 129.6     | 9.351     | 9.673     | 0.04451   |
하이퍼파라미터: {'depth': 7, 'learning_rate': 0.016393245237809825, 'l2_lea

In [22]:
max_params = optimizer.max['params']
max_params['depth'] = int(round(max_params['depth']))
max_params['border_count'] = int(round(max_params['border_count']))
max_params.update(fixed_params)

In [23]:
max_params

{'bagging_temperature': 0.8125525342743981,
 'border_count': 150,
 'depth': 7,
 'l2_leaf_reg': 9.330369744633948,
 'learning_rate': 0.016393245237809825,
 'loss_function': 'Logloss',
 'eval_metric': 'AUC',
 'iterations': 2500,
 'random_seed': 1991,
 'verbose': 200}

In [17]:
max_params_catboost = {
	'bagging_temperature': 0.8125525342743981,
	'border_count': 150,
	'depth': 7,
	'l2_leaf_reg': 9.330369744633948,
	'learning_rate': 0.016393245237809825,
	'loss_function': 'Logloss',
	'eval_metric': 'AUC',
	'iterations': 2500,
	'random_seed': 1991,
	'verbose': 200
}

In [19]:
from sklearn.metrics import roc_auc_score

# OOF 방식으로 모델 훈련, 검증, 예측
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

for idx, (train_idx, valid_idx) in enumerate(folds.split(X, y)):
    print('#'*40, f'폴드 {idx+1} / 폴드 {folds.n_splits}', '#'*40)
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    dtrain = Pool(X_train, y_train, cat_features=None)
    dvalid = Pool(X_valid, y_valid, cat_features=None)
    
    cat_model = CatBoostClassifier(**max_params_catboost)
    cat_model.fit(dtrain, eval_set=dvalid, early_stopping_rounds=200, verbose=500)
    
    # 테스트 데이터 예측
    oof_test_preds += cat_model.predict_proba(X_test)[:, 1] / folds.n_splits
    # 검증 데이터 예측
    oof_val_preds[valid_idx] += cat_model.predict_proba(X_valid)[:, 1]
    
    # 검증 데이터 ROC-AUC 계산
    roc_auc = roc_auc_score(y_valid, oof_val_preds[valid_idx])
    print(f'폴드 {idx+1} ROC-AUC : {roc_auc}\n')

######################################## 폴드 1 / 폴드 10 ########################################
0:	test: 0.7114529	best: 0.7114529 (0)	total: 46.9ms	remaining: 1m 57s
500:	test: 0.7377775	best: 0.7377775 (500)	total: 20.5s	remaining: 1m 21s
1000:	test: 0.7393023	best: 0.7393062 (989)	total: 40s	remaining: 59.9s
1500:	test: 0.7397951	best: 0.7397995 (1497)	total: 59.4s	remaining: 39.5s
2000:	test: 0.7399660	best: 0.7399982 (1979)	total: 1m 17s	remaining: 19.3s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.7399981714
bestIteration = 1979

Shrink model to first 1980 iterations.
폴드 1 ROC-AUC : 0.7389184648322544

######################################## 폴드 2 / 폴드 10 ########################################
0:	test: 0.7130583	best: 0.7130583 (0)	total: 41.1ms	remaining: 1m 42s
500:	test: 0.7354019	best: 0.7354641 (491)	total: 20.4s	remaining: 1m 21s
1000:	test: 0.7370967	best: 0.7371347 (998)	total: 39.1s	remaining: 58.5s
1500:	test: 0.7373972	best: 0.7374551 (1439)	to

## 5. 최종 결과

In [20]:
print('최종 OOF 검증 데이터 ROC-AUC:', roc_auc_score(y, oof_val_preds))

최종 OOF 검증 데이터 ROC-AUC: 0.7388998654317461


In [21]:
submission['probability'] = oof_test_preds
submission.to_csv('catboost_submission.csv', index=False)

In [22]:
submission = pd.read_csv('/kaggle/working/catboost_submission.csv')
submission.head()

,probability
0,0.001984
1,0.008637
2,0.183109
3,0.127837
4,0.495093
